<h2>This script is for internal developer testing.</h2>

In [ ]:
import wfdb

In [ ]:
import wfdb
# Testing rdsamp multiple samples per frame (and fmt 12)
sig, fields=wfdb.srdsamp('sampledata/03700181')
print(sig[:20,:])
print("\n\n", sig.shape, "\n\n")
print(fields)

print("\n\n\n")
print(sig[-20:,:])

In [ ]:
# Testing rdsamp format 80
sig, fields=wfdb.srdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
sig, fields=wfdb.srdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# For writing test annotation files based on first 3 annotations of 12726.anI

f=open('tmp.fake', 'wb')

indata= np.array([0, 236,   1,   0, 200,  84,   0,  88, 255, 251,  21, 252,  73, 110, 105, 116,
                  105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
                  112,   0,   0, 236,   0,   0,  67,  50,   0,  88,  21, 252,  67, 111, 110,  99,
108, 117, 100, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
112,   0,   0, 236,   0,   0, 114, 183,   0,  88,  23, 252,  73, 110, 105, 116,
105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 100,
111, 119, 110,   0, 0 ,0 ]).astype('u1')


print(indata)

indata.tofile(f)


f.close()


In [ ]:
# Trying to fix byte pointer for reading skewed signals. 

from wfdb import readsignal
import numpy as np
#sig, fields=readsignal.rdsamp('sampledata/03700181', channels=[0,2], sampfrom=1000, sampto=16000)
#sig=np.round(sig, decimals=8)
#targetsig=np.genfromtxt('tests/targetoutputdata/target10')

sig, fields=readsignal.rdsamp('sampledata/03700181', sampfrom=1000, sampto=16000, physical=0) # fs=125. -f 8 -t 128

print("\nsig: ", sig)
print('\n\n')
print("End of sig: ", sig[-10:,:])


# nbytesread = 15000 frames * 6 samples/frame * 1.5 bytes/sample = 135000

# Try to octal dump the right section of the file. After 16000 frames x (6 samples/frame)  = 96000 samples * (1.5 bytes per sample)
# = 144000 bytes skip. 



# od -t u1 03700181.dat -j 144000 -N 30 

# Result: 
# 236 255 236   2 240 236 163  27  61 236 255 236 236  15   2 147
#  27  40 236 255 236 236 255 236 130  27  19   2 240 236




In [ ]:
import wfdb
ann = wfdb.rdann('/home/chen/Downloads/chb01_03.edf','seizures')

In [ ]:
ann.__dict__

In [ ]:
# Testing wrann
import wfdb
from IPython.display import display

annsamp = [20, 40, 60, 80, 100, 800, 1200 , 3000, 3001, 8000, 100000]
#annsamp = [20, 40, 60, 80, 100, 800, 1200]
#annsamp = [20, 60, 2**31] # 2**31+100 is too large

nsamp = len(annsamp)

anntype = ['N', 'N', 'N', '3', 'N', '2', 'N', '1', 'N', 'N', 'g']
sub = [1,0,0,0,5,3,2,5,0,0,0]
chan = [2,2,3,2,9,2,2,2,2,2,2]
num = [1,2,3,4,5,3,2,5,11,22,22]
aux = [None, None, "good", "good", "good", "good", "good", "good", None, None, None]
fs = 100
fs = None

a = wfdb.Annotation(recordname = 'firstann', annotator = 'atr', 
                    annsamp = annsamp, anntype = anntype, aux = aux, 
                    subtype = sub, chan = chan, num = num, fs = fs)
a.type2aux()
display(a.__dict__)
a.wrann()

b = wfdb.rdann('firstann', 'atr')
b.__dict__


# print([sd & 255, sd & 768, (sd & 768) >> 8])

In [ ]:
r = requests.get('http://physionet.org/physiobank/database/mitdb/100.hea')
help(r.raise_for_status)

In [ ]:
with open('temp', "wb") as writefile:
                writefile.write(b'22')

In [ ]:
import wfdb
wfdb.dldatabase(pbdb = 'fecgsyndb', dlbasedir = '/home/chen/Downloads/dbfolder')